In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord

import os
import sep
import numpy as np
import pandas as pd
from scipy.spatial import KDTree
from scipy.interpolate import interp1d

from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.patches import Ellipse




In [ ]:
### my own functions
objname = "yasone1"
obsname = "0001"
sigma = 1

In [ ]:
###################################
# Substitute with your own directory
###################################

if int(obsname) == 1:
    date_g = date_r = date_i = "20230708"
elif int(obsname) == 2:
    date_g = date_r = "20230708"
    date_i = "20230709"
elif int(obsname) == 3:
    date_g = date_r = date_i = "20230709"
elif int(obsname) <= 6:
    date_g = date_r = date_i = "20230709"
elif int(obsname) == 7:
    date = "20230724"
else:
    date = "20230816"

fits_file_g = fits.open(f'../imaging/julen_stacked//GTC73-23A_{obsname}_Sloan_g_{date_g}_NOSKY_BBI.fits')
fits_file_r = fits.open(f'../imaging/julen_stacked//GTC73-23A_{obsname}_Sloan_r_{date_r}_NOSKY_BBI.fits')
fits_file_i = fits.open(f'../imaging/julen_stacked//GTC73-23A_{obsname}_Sloan_i_{date_i}_NOSKY_BBI.fits')


In [ ]:
image_data_g = fits_file_g[0].data
header_g = fits_file_g[0].header

image_data_r = fits_file_r[0].data
header_r = fits_file_r[0].header

image_data_i = fits_file_i[0].data
header_i = fits_file_i[0].header


In [ ]:
###################################
# I got an error sometimes when 
# trying to display the data.
# This should solve it
###################################

if image_data_g.dtype.byteorder not in ('=', '|'):
    image_data_g = image_data_g.astype(image_data_g.dtype.newbyteorder("="))
    
if image_data_r.dtype.byteorder not in ('=', '|'):
    image_data_r = image_data_r.astype(image_data_r.dtype.newbyteorder("="))
    
if image_data_i.dtype.byteorder not in ('=', '|'):
    image_data_i = image_data_i.astype(image_data_i.dtype.newbyteorder("="))



In [ ]:
#######################################
# Here I subtract the background noise
#######################################

image_data_contiguous_g = np.ascontiguousarray(image_data_g)
image_data_contiguous_r = np.ascontiguousarray(image_data_r)
image_data_contiguous_i = np.ascontiguousarray(image_data_i)

bkg_g = sep.Background(image_data_contiguous_g)
bkg_r = sep.Background(image_data_contiguous_r)
bkg_i = sep.Background(image_data_contiguous_i)

image_data_bkg_subtracted_g = image_data_contiguous_g - bkg_g
image_data_bkg_subtracted_r = image_data_contiguous_r - bkg_r
image_data_bkg_subtracted_i = image_data_contiguous_i - bkg_i


In [ ]:
objects_g = sep.extract(image_data_bkg_subtracted_g, sigma, err = bkg_g.globalrms)
objects_r = sep.extract(image_data_bkg_subtracted_r, sigma, err = bkg_r.globalrms)
objects_i = sep.extract(image_data_bkg_subtracted_i, sigma, err = bkg_i.globalrms)

df_g = pd.DataFrame(objects_g)
df_r = pd.DataFrame(objects_r)
# new
df_r["id"] = np.arange(len(df_r))
df_i = pd.DataFrame(objects_i)


In [ ]:
df_g['fwhm'] = 2.0 * np.sqrt(2.0 * np.log(2.0)) * np.sqrt((df_g['a']**2 + df_g['b']**2) / 2.0)
df_r['fwhm'] = 2.0 * np.sqrt(2.0 * np.log(2.0)) * np.sqrt((df_r['a']**2 + df_r['b']**2) / 2.0)
df_i['fwhm'] = 2.0 * np.sqrt(2.0 * np.log(2.0)) * np.sqrt((df_i['a']**2 + df_i['b']**2) / 2.0)

df_g['ellipticity'] = 1 - (df_g['b'] / df_g['a'])
df_r['ellipticity'] = 1 - (df_r['b'] / df_r['a'])
df_i['ellipticity'] = 1 - (df_i['b'] / df_i['a'])

flux_g, fluxerr_g, flag_g = sep.sum_circle(image_data_bkg_subtracted_g, objects_g['x'], objects_g['y'], 
                                           1.7*df_g['fwhm'], err = bkg_g.globalrms, gain = 1.0)

flux_r, fluxerr_r, flag_r = sep.sum_circle(image_data_bkg_subtracted_r, objects_r['x'], objects_r['y'], 
                                           1.7*df_r['fwhm'], err = bkg_r.globalrms, gain = 1.0)

flux_i, fluxerr_i, flag_i = sep.sum_circle(image_data_bkg_subtracted_i, objects_i['x'], objects_i['y'], 
                                           1.7*df_i['fwhm'], err = bkg_i.globalrms, gain = 1.0)


In [ ]:
plt.scatter(fluxerr_r, np.sqrt(flux_r), s=1, alpha=0.01)
plt.xscale("log")
plt.yscale("log")


In [ ]:
extinction_coefficient_g = 0.15 
extinction_coefficient_r = 0.07 
extinction_coefficient_i = 0.04 

extinction_coefficient_error_g = 0.02
extinction_coefficient_error_r = 0.01
extinction_coefficient_error_i = 0.01

airmass_array_g = [1.095, 1.090, 1.085, 1.080, 1.075]
airmass_array_r = [1.071, 1.067, 1.063, 1.060, 1.057]
airmass_array_i = [1.054, 1.051, 1.049, 1.046, 1.045]

airmass_g = np.mean(airmass_array_g)
airmass_r = np.mean(airmass_array_r)
airmass_i = np.mean(airmass_array_i)

airmass_error_g = np.std(airmass_array_g)
airmass_error_r = np.std(airmass_array_r)
airmass_error_i = np.std(airmass_array_i)


In [ ]:
if objname == "yasone1":
    zero_point_estimation_g = 28.15227
    zero_point_estimation_error_g = 0.01692 
    
    zero_point_estimation_r = 28.24766 
    zero_point_estimation_error_r = 0.01008
    
    zero_point_estimation_i = 27.81407
    zero_point_estimation_error_i = 0.01047
    ra0, dec0 = 265.52019, 13.17146


elif objname == "yasone2":
    zero_point_estimation_g = 28.12761 
    zero_point_estimation_error_g = 0.01931
    
    zero_point_estimation_r = 28.13342
    zero_point_estimation_error_r = 0.00882
    
    zero_point_estimation_i = 27.61163
    zero_point_estimation_error_i = 0.01528
    ra0, dec0 = 262.34921, 6.42302

elif objname == "yasone3":
    
    zero_point_estimation_g = 27.93537
    zero_point_estimation_error_g = 0.01880
    
    zero_point_estimation_r = 28.06096 
    zero_point_estimation_error_r = 0.01128
    
    zero_point_estimation_i = 27.49029
    zero_point_estimation_error_i = 0.01095
    ra0, dec0 = 292.96258, -26.44994


In [ ]:
###################################
# Here is where we finally obtain flux 
# and magnitude values
###################################

df_g['mag_inst'] = -2.5 * np.log10(flux_g)
df_r['mag_inst'] = -2.5 * np.log10(flux_r)
df_i['mag_inst'] = -2.5 * np.log10(flux_i)

df_g['mag_inst_err'] = (2.5 / np.log(10)) * (fluxerr_g / flux_g)
df_r['mag_inst_err'] = (2.5 / np.log(10)) * (fluxerr_r / flux_r)
df_i['mag_inst_err'] = (2.5 / np.log(10)) * (fluxerr_i / flux_i)

df_g['mag_apparent'] = zero_point_estimation_g + df_g['mag_inst'] - (extinction_coefficient_g * airmass_g)
df_r['mag_apparent'] = zero_point_estimation_r + df_r['mag_inst'] - (extinction_coefficient_r * airmass_r)
df_i['mag_apparent'] = zero_point_estimation_i + df_i['mag_inst'] - (extinction_coefficient_i * airmass_i)

df_g['mag_apparent_error'] = np.sqrt(zero_point_estimation_error_g**2 + df_g['mag_inst_err']**2 +
                                    (airmass_g * extinction_coefficient_error_g)**2 + 
                                    (extinction_coefficient_g * airmass_error_g)**2)

df_r['mag_apparent_error'] = np.sqrt(zero_point_estimation_error_r**2 + df_r['mag_inst_err']**2 +
                                    (airmass_r * extinction_coefficient_error_r)**2 + 
                                    (extinction_coefficient_r * airmass_error_r)**2)

df_i['mag_apparent_error'] = np.sqrt(zero_point_estimation_error_i**2 + df_i['mag_inst_err']**2 +
                                    (airmass_i * extinction_coefficient_error_i)**2 + 
                                    (extinction_coefficient_i * airmass_error_i)**2)


In [ ]:
# Get the World Coordinate System information

wcs_g = WCS(header_g)
wcs_r = WCS(header_r)
wcs_i = WCS(header_i)

# Convert pixel coordinates to world coordinates (RA, DEC)
# The second argument is the origin (1 for FITS convention)

ra_dec_g = wcs_g.wcs_pix2world(np.column_stack((df_g['x'], df_g['y'])), 1)
df_g['ra'] = ra_dec_g[:, 0]
df_g['dec'] = ra_dec_g[:, 1]

ra_dec_r = wcs_r.wcs_pix2world(np.column_stack((df_r['x'], df_r['y'])), 1)
df_r['ra'] = ra_dec_r[:, 0]
df_r['dec'] = ra_dec_r[:, 1]

ra_dec_i = wcs_i.wcs_pix2world(np.column_stack((df_i['x'], df_i['y'])), 1)
df_i['ra'] = ra_dec_i[:, 0]
df_i['dec'] = ra_dec_i[:, 1]


In [ ]:
###################################
# Here I just did a little zoom to focus 
# on the region around our candidate's centroid. 
# This step can of course take place much 
# earlier or later, to your liking.
# For Yasone 2 and 3, their centroids are in the paper.
# These centroids were determined when I was searching for overdensities
# in Pan-STARRS, and they just happened to be the coordinates of the stellar
# member with the highest number of stellar neighbours, but of course
# this centroid can be adjusted. I leave it to your criteria. 
###################################

df_cut_g = df_g[(df_g['ra'] >= ra0 - 0.5) & (df_g['ra'] <= ra0 + 0.5) &
                (df_g['dec'] >= dec0 - 0.5) & (df_g['dec'] <= dec0 + 0.5)].copy()

df_cut_r = df_r[(df_r['ra'] >= ra0 - 0.5) & (df_r['ra'] <= ra0 + 0.5) &
                (df_r['dec'] >= dec0 - 0.5) & (df_r['dec'] <= dec0 + 0.5)].copy()

df_cut_i = df_i[(df_i['ra'] >= ra0 - 0.5) & (df_i['ra'] <= ra0 + 0.5) &
                (df_i['dec'] >= dec0 - 0.5) & (df_i['dec'] <= dec0 + 0.5)].copy()


# p1_cut = p1_cut.reset_index(drop = True)
df_cut_g = df_cut_g.reset_index(drop = True)
df_cut_r = df_cut_r.reset_index(drop = True)
df_cut_i = df_cut_i.reset_index(drop = True)


In [ ]:
###################################
# Now that we have 3 different dataframes with 
# a varying number of sources 
# (depending on the threshold value, the definition of the aperture etc.)
# this is how I would typically "join them" so that we can evaluate all 
# the bands information for each source. Since I was especially interested
# in recreating g vs r-g and r vs r-i plots, I created 2 dataframes, one containing
# the information from the g and r bands, and another one with r and i.
###################################

observed_coords_g = df_cut_g[['ra', 'dec']].to_numpy()
observed_coords_r = df_cut_r[['ra', 'dec']].to_numpy()
observed_coords_i = df_cut_i[['ra', 'dec']].to_numpy()

# Create KDTree for the bottleneck (the array with least amount of sources). In g-r that's g. In r-i that's r.
tree_g = KDTree(observed_coords_g)
tree_r = KDTree(observed_coords_r)

# Query the KDTree for the nearest neighbor of each observed coordinate. Here is where you specify the secoundary array (r, i)
distances_gr, indices_gr = tree_g.query(observed_coords_r, distance_upper_bound = 1/3600)  # 1 arcsecond = 1/3600 degree
distances_ri, indices_ri = tree_r.query(observed_coords_i, distance_upper_bound = 1/3600)

# Filter out matches within 1 arcsecond
valid_matches_gr = distances_gr < np.inf
valid_indices_gr = indices_gr[valid_matches_gr]
valid_distances_gr = distances_gr[valid_matches_gr]

valid_matches_ri = distances_ri < np.inf
valid_indices_ri = indices_ri[valid_matches_ri]
valid_distances_ri = distances_ri[valid_matches_ri]

# Build the dataframe of matched sources for g-r and r-i
# ¡¡¡IMPORTANT!!!
# WHEN EXTRACTING THE ROWS OF INTEREST FROM THE ORIGINAL BOTTLENECK DATAFRAME ==> USE valid_indices
# WHEN EXTRACTING THE ROWS OF INTEREST FROM THE ORIGINAL LARGER DATAFRAME ==> USE valid_matches
matched_sources_gr = pd.DataFrame({
    'ra_g': df_cut_g.iloc[valid_indices_gr]['ra'].values,
    'dec_g': df_cut_g.iloc[valid_indices_gr]['dec'].values,
    'mag_g': df_cut_g.iloc[valid_indices_gr]['mag_apparent'].values,
    'mag_g_err': df_cut_g.iloc[valid_indices_gr]['mag_apparent_error'].values,
    'ellipticity_g': df_cut_g.iloc[valid_indices_gr]['ellipticity'].values,
    'ra_r': df_cut_r.iloc[valid_matches_gr]['ra'].values,
    'dec_r': df_cut_r.iloc[valid_matches_gr]['dec'].values,
    'mag_r': df_cut_r.iloc[valid_matches_gr]['mag_apparent'].values,
    'mag_r_err': df_cut_r.iloc[valid_matches_gr]['mag_apparent_error'].values,
    'ellipticity_r': df_cut_r.iloc[valid_matches_gr]['ellipticity'].values,
    'distance_gr': valid_distances_gr,
    'id': df_cut_r.iloc[valid_matches_gr]['id'].values,
    'fwhm': df_cut_r.iloc[valid_matches_gr]['fwhm'].values,

})

matched_sources_gr['mag_gr'] = matched_sources_gr['mag_g'] - matched_sources_gr['mag_r']
matched_sources_gr['mag_gr_err'] = np.sqrt(matched_sources_gr['mag_g_err']**2 + matched_sources_gr['mag_r_err']**2)

matched_sources_ri = pd.DataFrame({
    'ra_r': df_cut_r.iloc[valid_indices_ri]['ra'].values,
    'dec_r': df_cut_r.iloc[valid_indices_ri]['dec'].values,
    'mag_r': df_cut_r.iloc[valid_indices_ri]['mag_apparent'].values,
    'mag_r_err': df_cut_r.iloc[valid_indices_ri]['mag_apparent_error'].values,
    'ellipticity_r': df_cut_r.iloc[valid_indices_ri]['ellipticity'].values,
    'ra_i': df_cut_i.iloc[valid_matches_ri]['ra'].values,
    'dec_i': df_cut_i.iloc[valid_matches_ri]['dec'].values,
    'mag_i': df_cut_i.iloc[valid_matches_ri]['mag_apparent'].values,
    'mag_i_err': df_cut_i.iloc[valid_matches_ri]['mag_apparent_error'].values,
    'ellipticity_i': df_cut_i.iloc[valid_matches_ri]['ellipticity'].values,
    'distance_ri': valid_distances_ri,
    'id': df_cut_r.iloc[valid_indices_ri]['id'].values,

})

matched_sources_ri['mag_ri'] = matched_sources_ri['mag_r'] - matched_sources_ri['mag_i']
matched_sources_ri['mag_ri_err'] = np.sqrt(matched_sources_ri['mag_r_err']**2 + matched_sources_ri['mag_i_err']**2)


In [ ]:
# Selecting sources from the candidate g-r
radius_g = np.sqrt((matched_sources_gr['ra_g'] - ra0)**2 + (matched_sources_gr['dec_g'] - dec0)**2)
radius_r = np.sqrt((matched_sources_gr['ra_r'] - ra0)**2 + (matched_sources_gr['dec_r'] - dec0)**2)
radius_gr = (radius_g + radius_r)/2

matched_sources_gr['radius'] = radius_gr
matched_sources_gr_cut = matched_sources_gr[matched_sources_gr['radius'] <= 40/3600].copy()



In [ ]:
###################################
# Here I apply the Tonry corrections
# to be able to plot in the same CMD
# stars from Pan-STARRS and GTC, Which use 
# slightly different filters
###################################


# transforming my G,R bands from GTC to the PanSTARRS notation of filters

matched_sources_gr_cut['mag_g_good'] = (matched_sources_gr_cut['mag_g'] - 0.011 - 0.125*(matched_sources_gr_cut['mag_gr']) 
                                       - 0.015 * (matched_sources_gr_cut['mag_gr']** 2))

matched_sources_gr_cut['mag_r_good'] = (matched_sources_gr_cut['mag_r'] + 0.001 - 0.006*(matched_sources_gr_cut['mag_gr'])
                                       - 0.002 * (matched_sources_gr_cut['mag_gr'] ** 2))

# Error propagation for mag_g_good

matched_sources_gr_cut['mag_g_good_err'] = np.sqrt(
    ((0.875 - 0.03 * matched_sources_gr_cut['mag_gr']) * matched_sources_gr_cut['mag_g_err']) ** 2 +
    ((-0.125 + 0.03 * matched_sources_gr_cut['mag_gr']) * matched_sources_gr_cut['mag_r_err']) ** 2 +
    (0.006**2) #Associated error of the conversion, Table 6 THE Pan-STARRS1 PHOTOMETRIC SYSTEM Tonry et al. 2012
)

# Error propagation for mag_r_good

matched_sources_gr_cut['mag_r_good_err'] = np.sqrt(
    ((-0.006 - 0.004 * matched_sources_gr_cut['mag_gr']) * matched_sources_gr_cut['mag_g_err']) ** 2 +
    ((1.006 - 0.004 * matched_sources_gr_cut['mag_gr']) * matched_sources_gr_cut['mag_r_err']) ** 2 +
    (0.002**2) #Associated error of the conversion, Table 6 THE Pan-STARRS1 PHOTOMETRIC SYSTEM Tonry et al. 2012
)

# Error propagation for mag_gr_good

matched_sources_gr_cut['mag_gr_good'] = matched_sources_gr_cut['mag_g_good'] - matched_sources_gr_cut['mag_r_good']

matched_sources_gr_cut['mag_gr_good_err'] = np.sqrt(
    matched_sources_gr_cut['mag_g_good_err']**2 + matched_sources_gr_cut['mag_r_good_err']**2)



# New analysis

In [ ]:
plt.scatter(matched_sources_gr["mag_g"] - matched_sources_gr["mag_r"], matched_sources_gr["mag_g"])
plt.xlim(-0.5, 2.0)
plt.ylim(26, 16)


In [ ]:
import arya

In [ ]:
plt.scatter(matched_sources_gr["ra_g"], matched_sources_gr["dec_g"], s=2)
plt.scatter(matched_sources_ri["ra_r"], matched_sources_ri["dec_r"], s=1)
plt.scatter(matched_sources_ri["ra_i"], matched_sources_ri["dec_i"], s=0.5)

In [ ]:
plt.scatter(matched_sources_gr_cut["mag_gr"], matched_sources_gr_cut["mag_g"])
plt.xlim(-0.5, 2.0)
plt.ylim(26, 16)

In [ ]:
plt.scatter(matched_sources_gr_cut["mag_gr_good"], matched_sources_gr_cut["mag_g_good"])
plt.xlim(-0.5, 2.0)
plt.ylim(26, 16)

In [ ]:
matched_sources_gr

In [ ]:
matched_sources_ri.set_index("id")["ra_i"]

In [ ]:
matched_sources_both = matched_sources_gr.join(matched_sources_ri.set_index("id")[["ra_i", "dec_i", "mag_i", "mag_i_err", "distance_ri", "mag_ri", "mag_ri_err"]], on="id", )
matched_sources_both

In [ ]:
matched_sources_both.rename(
   columns= {
    "mag_r": "RMAG",
    "mag_g": "GMAG",
    "mag_i": "IMAG",
    "ra_r": "ALPHA_J2000",
    "dec_r": "DELTA_J2000",
    
    
    }
)

In [ ]:
matched_sources_both.loc[matched_sources_both["distance_gr"] > 1/3600, "mag_g"] = np.nan
matched_sources_both.loc[matched_sources_both["distance_ri"] > 1/3600, "mag_i"] = np.nan

In [ ]:
matched_sources_both['mag_g_ps'] = (matched_sources_both['mag_g'] - 0.011 - 0.125*(matched_sources_both['mag_gr']) 
                                       - 0.015 * (matched_sources_both['mag_gr']** 2))

matched_sources_both['mag_r_ps'] = (matched_sources_both['mag_r'] + 0.001 - 0.006*(matched_sources_both['mag_gr'])
                                       - 0.002 * (matched_sources_both['mag_gr'] ** 2))

matched_sources_both['mag_i_ps'] = (matched_sources_both['mag_i'] + 0.004 - 0.014*(matched_sources_both['mag_gr'])
                                       - 0.001 * (matched_sources_both['mag_gr'] ** 2))

# Error propagation for mag_g_good

matched_sources_both['mag_g_ps_err'] = np.sqrt(
    ((0.875 - 0.03 * matched_sources_both['mag_gr']) * matched_sources_both['mag_g_err']) ** 2 +
    ((-0.125 + 0.03 * matched_sources_both['mag_gr']) * matched_sources_both['mag_r_err']) ** 2 +
    (0.006**2) #Associated error of the conversion, Table 6 THE Pan-STARRS1 PHOTOMETRIC SYSTEM Tonry et al. 2012
)

# Error propagation for mag_r_good

matched_sources_both['mag_r_ps_err'] = np.sqrt(
    ((-0.006 - 0.004 * matched_sources_both['mag_gr']) * matched_sources_both['mag_g_err']) ** 2 +
    ((1.006 - 0.004 * matched_sources_both['mag_gr']) * matched_sources_both['mag_r_err']) ** 2 +
    (0.002**2) #Associated error of the conversion, Table 6 THE Pan-STARRS1 PHOTOMETRIC SYSTEM Tonry et al. 2012
)

matched_sources_both['mag_i_ps_err'] = np.sqrt(
    ((0.004 - 0.024 * matched_sources_both['mag_gr']) * matched_sources_both['mag_g_err']) ** 2 +
    ((0.004 - 0.024 * matched_sources_both['mag_gr']) * matched_sources_both['mag_r_err']) ** 2 +
    + matched_sources_both['mag_i_err']**2 + 
    (0.003**2) #Associated error of the conversion, Table 6 THE Pan-STARRS1 PHOTOMETRIC SYSTEM Tonry et al. 2012
)

matched_sources_both['mag_gr_ps'] = matched_sources_both['mag_g_ps'] - matched_sources_both['mag_r_ps']

matched_sources_both['mag_gr_ps_err'] = np.sqrt(
    matched_sources_both['mag_g_ps_err']**2 + matched_sources_both['mag_r_ps_err']**2)

matched_sources_both['mag_ri_ps'] = matched_sources_both['mag_r_ps'] - matched_sources_both['mag_i_ps']

matched_sources_both['mag_ri_ps_err'] = np.sqrt(
    matched_sources_both['mag_r_ps_err']**2 + matched_sources_both['mag_i_ps_err']**2)


In [ ]:
matched_out = matched_sources_both.rename(
    columns={
    "mag_r": "RMAG",
    "mag_g": "GMAG",
    "mag_i": "IMAG",
    "mag_r_err": "RMAG_ERR",
    "mag_g_err": "GMAG_ERR",
    "mag_i_err": "IMAG_ERR",
    "ra_r": "ALPHA_J2000",
    "dec_r": "DELTA_J2000",
    
    
    }
)

In [ ]:
import astropy

In [ ]:
tab_out = astropy.table.Table.from_pandas(matched_out)

In [ ]:
tab_out["ALPHA_J2000"]

In [ ]:
pwd

In [ ]:
tab_out.write(f"{objname}_julen.cat", format="fits", overwrite=True)

In [ ]:
plt.scatter(matched_sources_both["mag_g"], matched_sources_both["mag_g_ps"] - matched_sources_both["mag_g"])

In [ ]:
plt.scatter(matched_sources_both["mag_g_ps"] - matched_sources_both["mag_r_ps"], matched_sources_both["mag_g_ps"])
plt.scatter(matched_sources_both["mag_g"] - matched_sources_both["mag_r"], matched_sources_both["mag_g"])
plt.xlim(-0.5, 2)
plt.ylim(25, 16)

In [ ]:
plt.scatter(matched_sources_both["mag_r_ps"] - matched_sources_both["mag_i_ps"], matched_sources_both["mag_r_ps"])
plt.scatter(matched_sources_both["mag_r"] - matched_sources_both["mag_i"], matched_sources_both["mag_r"])
plt.xlim(-0.5, 2)
plt.ylim(25, 16)

In [ ]:
bkg_g.globalrms, bkg_r.globalrms, bkg_i.globalrms

In [ ]:
 2**16 / np.max(image_data_g) * bkg_g.globalrms

In [ ]:
 2**16 / np.max(image_data_r) * bkg_r.globalrms

In [ ]:
 2**16 / np.max(image_data_i) * bkg_i.globalrms

In [ ]:
np.median(df_g["fwhm"])

In [ ]:
np.median(df_r["fwhm"])

In [ ]:
np.median(df_i["fwhm"])